In [79]:
from transformers import Wav2Vec2FeatureExtractor, HubertModel
import torchaudio
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import os
import pickle
import requests
import sounddevice as sd
from scipy.io.wavfile import write
import threading
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

In [80]:
# HuBERT 특성 추출기
class HuBERTFeatureExtractor:
    def __init__(self, model_name="facebook/hubert-base-ls960"):
        self.processor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
        self.model = HubertModel.from_pretrained(model_name)
        self.model.eval()

    def load_audio(self, audio_file):
        waveform, sample_rate = torchaudio.load(audio_file)
        return waveform, sample_rate

    def preprocess_audio(self, waveform, sample_rate, target_sample_rate=16000, max_length=10):
        if waveform.size(0) > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        if sample_rate != target_sample_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
            waveform = resampler(waveform)

        max_samples = target_sample_rate * max_length
        if waveform.size(1) > max_samples:
            waveform = waveform[:, :max_samples]

        return waveform

    def extract_features(self, audio_file):
        waveform, sample_rate = self.load_audio(audio_file)
        waveform = self.preprocess_audio(waveform, sample_rate)

        if waveform.dim() == 1:
            waveform = waveform.unsqueeze(0)
        elif waveform.dim() == 2:
            if waveform.size(0) > 1:
                waveform = waveform.mean(dim=0, keepdim=True)
        elif waveform.dim() == 3:
            waveform = waveform.squeeze(0)
            if waveform.size(0) > 1:
                waveform = waveform.mean(dim=0, keepdim=True)

        inputs = self.processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
        input_values = inputs.input_values

        input_values = input_values.squeeze(1)

        with torch.no_grad():
            outputs = self.model(input_values)

        features = outputs.last_hidden_state

        return features

In [81]:
# 감정 데이터셋 클래스
class EmotionDataset(Dataset):
    def __init__(self, audio_files, labels, feature_extractor, label_encoder_path):
        self.audio_files = audio_files
        self.labels = labels
        self.feature_extractor = feature_extractor
        self.label_encoder = LabelEncoder()

        if label_encoder_path and os.path.exists(label_encoder_path):
            with open(label_encoder_path, 'rb') as f:
                self.label_encoder = pickle.load(f)
            print(f"LabelEncoder loaded from {label_encoder_path}")
        else:
            self.label_encoder.fit(labels)
            if label_encoder_path:
                with open(label_encoder_path, 'wb') as f:
                    pickle.dump(self.label_encoder, f)
            print(f"LabelEncoder saved to {label_encoder_path}")

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio_file = self.audio_files[idx]
        label = self.labels[idx]

        features = self.feature_extractor.extract_features(audio_file)
        features = features.squeeze(0)

        label = self.label_encoder.transform([label])[0]

        return features, label

In [82]:
# Transformer 모델 (감정 인식용)
class EmotionTransformer(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(EmotionTransformer, self).__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=input_dim, nhead=16),
            num_layers=8
        )
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x, src_key_padding_mask=None):
        x = self.transformer(x, src_key_padding_mask=src_key_padding_mask)
        x = x.mean(dim=0)  # 평균을 사용
        output = self.fc(x)
        return output

In [83]:
# 테스트 함수 정의
def test_model(test_dataloader, model, dataset):
    model.eval()
    predictions = []
    ground_truth = []

    with torch.no_grad():
        for features, labels in test_dataloader:
            features = features.to(device)
            labels = labels.to(device)

            # 모델에 입력
            features = features.permute(1, 0, 2)

            outputs = model(features)

            predicted_labels = torch.argmax(outputs, dim=1)
            predictions.extend(predicted_labels.cpu().tolist())
            ground_truth.extend(labels.cpu().tolist())

    # 예측 결과 반환
    return predictions, ground_truth

In [84]:
def collate_fn(batch):
    features = [item[0] for item in batch]  # 각 샘플의 feature 추출
    labels = torch.tensor([item[1] for item in batch])  # 각 샘플의 label 추출
    
    # 각 feature의 크기 확인 후, tensor로 변환
    features = [f.squeeze(0) if len(f.shape) == 3 else f for f in features]
    
    # 시퀀스 패딩 적용 (seq_len을 동일하게 맞춤)
    features = pad_sequence(features, batch_first=True, padding_value=0)  # (batch_size, max_seq_len, feature_dim)
    
    return features, labels

In [85]:
# TMDB API 키
TMDB_API_KEY = "df9a0caaf2a07ee6babd7024a6accaf8"
    
EMOTION_TO_GENRE = {
    '기쁨': 35,  # Comedy
    '슬픔': 18,  # Drama
    '분노': 53,  # Thriller
    '불안': 27,  # Horror
    '상처': 80,  # Crime
    '당황': 28,  # Action
    '중립': 10751,  # Family
}

def get_recommendations(emotion, result_num=10, api_key=TMDB_API_KEY):
    # 감정 매핑 확인
    genre_id = EMOTION_TO_GENRE.get(emotion)
    if not genre_id:
        return f"'{emotion}'에 해당하는 추천 장르가 없습니다. 감정을 다시 입력해주세요."

    # TMDB Discover API 호출
    url = f"https://api.themoviedb.org/3/discover/movie"
    params = {
        "api_key": api_key,
        "include_video": True,
        "with_genres": genre_id,
        "sort_by": "popularity.desc",  # 인기 순으로 정렬
        "language": "ko-KR",          # 한국어 결과
        "vote_average.gte": 7.0,      # 평점 7 이상
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        return f"TMDB API 호출 실패: {response.status_code}"

    data = response.json()
    results = data.get("results", [])

    if not results:
        return f"'{emotion}'에 맞는 추천 콘텐츠를 찾을 수 없습니다."

    # 추천 콘텐츠 추출
    recommendations = []
    for movie in results[:result_num]:  # 상위 N개만 추출
        recommendations.append({
            "title": movie.get("title"),
            "overview": movie.get("overview"),
            "vote_average": movie.get("vote_average"),
            "release_date": movie.get("release_date"),
        })

    return recommendations

In [72]:
# 설정
fs = 44100  # 샘플링 레이트
output_filename = "./record files/output.wav"  # 저장할 파일 이름
stop_recording = False  # 녹음 중단 플래그
seconds = 10 # 녹음 시간(초)


def record_audio(): # 마이크로 음성을 녹음하는 함수.
    global stop_recording, audio_data
    print("녹음 시작. 'Enter' 키를 누르면 녹음을 멈춥니다.")
    audio_data = sd.rec(int(60 * fs), samplerate=fs, channels=1, dtype='int16')  # 최대 60초 녹음
    while not stop_recording:
        sd.sleep(100)  # 짧은 대기(0.1초)
    sd.stop()  # 녹음 중단
    print("녹음 중단 중...")


def wait_for_stop(): # 사용자가 'Enter' 키를 누를 때까지 대기.
    global stop_recording
    while not stop_recording:
        command = input("입력: ")
        if command.strip().lower() == "":
            stop_recording = True


# 스레드 생성 및 실행
recording_thread = threading.Thread(target=record_audio)
input_thread = threading.Thread(target=wait_for_stop)

recording_thread.start()
input_thread.start()

recording_thread.join()
input_thread.join()

# 녹음 데이터를 파일로 저장
write(output_filename, fs, audio_data[:fs * seconds])
print(f"'{output_filename}' 파일로 저장되었습니다.")


녹음 시작. 'Enter' 키를 누르면 녹음을 멈춥니다.
녹음 중단 중...
'./record files/output.wav' 파일로 저장되었습니다.


In [86]:
folder_name = "2025-01-13"

In [87]:
### 정확도 계산(테스트)

# 테스트 데이터 준비
test_audio_files = ["./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/1.기쁨/0029_G2A4E1S0C0_KJE/0029_G2A4E1S0C0_KJE_001970.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/2.슬픔/0033_G2A3E2S0C0_KMA/0033_G2A3E2S0C0_KMA_000020.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/3.분노/0018_G2A3E3S0C0_JBR/0018_G2A3E3S0C0_JBR_000019.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/4.불안/0012_G1A2E4S0C0_CHY/0012_G1A2E4S0C0_CHY_000011.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/5.상처/0005_G1A3E5S0C0_LJB/0005_G1A3E5S0C0_LJB_000014.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/6.당황/0020_G2A4E6S0C0_HGW/0020_G2A4E6S0C0_HGW_000009.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/7.중립/0044_G2A5E7S0C0_KTH/0044_G2A5E7S0C0_KTH_000012.wav"]  # 테스트 오디오 파일 경로 리스트
test_labels = ["기쁨", "슬픔", "분노", "불안", "상처", "당황", "중립"]  # 테스트 라벨 리스트

# HuBERT 특성 추출기와 LabelEncoder 로드
feature_extractor = HuBERTFeatureExtractor()
label_encoder_path = f"./model, label encoder/{folder_name}/label_encoder.pkl"
dataset = EmotionDataset(test_audio_files, test_labels, feature_extractor, label_encoder_path)
test_dataloader = DataLoader(dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# 모델 불러오기
model_path = f"./model, label encoder/{folder_name}/model.pth"
device = torch.device("mps")
model = torch.load(model_path)
model = model.to(device)

# 테스트 실행
predictions, ground_truth = test_model(test_dataloader, model, dataset)

# 테스트 결과 출력
predicted_labels = dataset.label_encoder.inverse_transform(predictions)
ground_truth_labels = dataset.label_encoder.inverse_transform(ground_truth)

# 정확도 계산
correct_predictions = sum([1 for p, g in zip(predictions, ground_truth) if p == g])
accuracy = correct_predictions / len(predictions)

# 결과 출력
print(f"Predictions: {predicted_labels}")
print(f"Ground Truth: {ground_truth_labels}")
print(f"Accuracy: {accuracy:.4f}")

LabelEncoder loaded from ./model, label encoder/2025-01-13/label_encoder.pkl


/var/folders/4h/8jkvbs3n7tlcqj21nw5kbbqm0000gn/T/ipykernel_35765/3425460078.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


Predictions: ['기쁨' '슬픔' '분노' '분노' '슬픔' '불안' '중립']
Ground Truth: ['기쁨' '슬픔' '분노' '불안' '상처' '당황' '중립']
Accuracy: 0.5714


In [74]:
### 정확도 계산 X(예측만)

# 테스트 데이터 준비
test_audio_files = ["./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/1.기쁨/0029_G2A4E1S0C0_KJE/0029_G2A4E1S0C0_KJE_001970.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/2.슬픔/0033_G2A3E2S0C0_KMA/0033_G2A3E2S0C0_KMA_000020.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/3.분노/0018_G2A3E3S0C0_JBR/0018_G2A3E3S0C0_JBR_000019.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/4.불안/0012_G1A2E4S0C0_CHY/0012_G1A2E4S0C0_CHY_000011.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/5.상처/0005_G1A3E5S0C0_LJB/0005_G1A3E5S0C0_LJB_000014.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/6.당황/0020_G2A4E6S0C0_HGW/0020_G2A4E6S0C0_HGW_000009.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/7.중립/0044_G2A5E7S0C0_KTH/0044_G2A5E7S0C0_KTH_000012.wav"]  # 테스트 오디오 파일 경로 리스트
test_labels = ["기쁨", "슬픔", "분노", "불안", "상처", "당황", "중립"]  # 테스트 라벨 리스트

# HuBERT 특성 추출기와 LabelEncoder 로드
feature_extractor = HuBERTFeatureExtractor()
label_encoder_path = f"./model, label encoder/{folder_name}/label_encoder.pkl"
dataset = EmotionDataset(test_audio_files, test_labels, feature_extractor, label_encoder_path)
test_dataloader = DataLoader(dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# 모델 불러오기
model_path = f"./model, label encoder/{folder_name}/model.pth"
device = torch.device("mps")
model = torch.load(model_path)
model = model.to(device)

# 정답 레이블 입력 X (예측만)
# 테스트 실행
predictions, _ = test_model(test_dataloader, model, dataset)

# 예측값을 리버스 인코딩하여 감정 레이블로 변환
predicted_labels = dataset.label_encoder.inverse_transform(predictions)

# 결과 출력
print(f"Predictions: {predicted_labels}")

LabelEncoder loaded from ./model, label encoder/2025-01-13/label_encoder.pkl


/var/folders/4h/8jkvbs3n7tlcqj21nw5kbbqm0000gn/T/ipykernel_35765/412774034.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


Predictions: ['기쁨' '슬픔' '분노' '분노' '슬픔' '불안' '중립']


In [92]:
video_link = {'나쁜 녀석들: 라이드 오어 다이': 'https://youtu.be/_COstzwNXxc?si=YLovPvRmImAr1QIv', '수퍼 소닉 3': 'https://youtu.be/ngdTcr1FaDY?si=BY7uQyB6JllfCSu-'}

In [93]:
for i in predicted_labels:
    recommendations = get_recommendations(i, 3)

    if isinstance(recommendations, str):
        print(recommendations)  # 에러 메시지 출력
    else:
        print(f"'{i}'에 맞는 추천 콘텐츠:")
        for idx, movie in enumerate(recommendations, 1):
            print(f"\n{idx}. 제목: {movie['title']}")
            print(f"   개봉일: {movie['release_date']}")
            print(f"   평점: {movie['vote_average']}")
            print(f"   줄거리: {movie['overview']}")
            print(f"   비디오 링크: {video_link.get(movie['title'], '없음')}")

'기쁨'에 맞는 추천 콘텐츠:

1. 제목: 수퍼 소닉 3
   개봉일: 2024-12-19
   평점: 7.848
   줄거리: 너클즈, 테일즈와 함께 평화로운 일상을 보내던 초특급 히어로 소닉. 연구 시설에 50년간 잠들어 있던 사상 최강의 비밀 병기 "섀도우"가 탈주하자, 세계 수호 통합 부대(약칭 세.수.통)에 의해 극비 소집된다. 소중한 것을 잃은 분노와 복수심에 불타는 섀도우는 소닉의 초고속 스피드와 너클즈의 최강 펀치를 단 단숨에 제압해버린다. 세상을 지배하려는 닥터 로보트닉과 그의 할아버지 제럴드 박사는 섀도우의 엄청난 힘 카오스 에너지를 이용해 인류를 정복하려고 하는데…
   비디오 링크: https://youtu.be/ngdTcr1FaDY?si=BY7uQyB6JllfCSu-

2. 제목: 모아나 2
   개봉일: 2024-11-21
   평점: 7.2
   줄거리: 바다를 누볐던 선조들에게서 예기치 못한 부름을 받은 모아나가 마우이와 다시 만나 새로운 선원들과 함께 오랫동안 잊혀진 멀고 위험한 바다 너머로 떠나는 특별한 모험을 담은 이야기
   비디오 링크: 없음

3. 제목: 아노라
   개봉일: 2024-10-14
   평점: 7.1
   줄거리: 뉴욕의 스트리퍼 아노라는 자신의 바를 찾은 철부지 러시아 재벌2세 이반을 만나게 되고 충동적인 사랑을 믿고 허황된 신분 상승을 꿈꾸며 결혼식을 올리게 된다. 그러나 신데렐라 스토리를 꿈꿨던 것도 잠시, 한 번도 본 적 없는 이반의 부모님이 아들의 결혼 사실을 알게 되자 길길이 날뛰며 미국에 있는 하수인 3인방에게 둘을 잡아 혼인무효소송을 진행할 것을 지시한다. 하수인 3인이 들이닥치자 부모님이 무서워 겁에 질린 남편 이반은 아노라를 버린채 홀로 도망친다. 이반을 찾아 결혼 생활을 유지하고 싶은 아노라와 어떻게든 이반을 찾아 혼인무효소송을 시켜야만 하는 하수인 3인방의 대환장 발악이 시작된다.
   비디오 링크: 없음
'슬픔'에 맞는 추천 콘텐츠:

1. 제목: Nr. 24
   개봉일: 2024